In [9]:
import pandas as pd
import plotly.graph_objects as go
import json
import base64
import io
import os

# Define the sample pandas DataFrame
data = [
     {
        "GRANULARITY": "YTD-2023",
        "BD_SAVING_ACT_PER_KG": 86.07359974848711,
        "RM_PRICE_RP_PER_KG": 313.80201783724624,
        "RM_PORTO_SAVING_PER_KG": -760.8857730115999,
        "PM_PRICE_RP_PER_KG": 74.7897848595967,
        "PM_PORTO_SAVING_PER_KG": -43.58483013488169,
        "OPERATIONAL_COST_RP_PER_KG": 585.679768339234
     },
     {
        "GRANULARITY": "Jan-2023",
        "BD_SAVING_ACT_PER_KG": 86.07359974848711,
        "RM_PRICE_RP_PER_KG": 313.80201783724624,
        "RM_PORTO_SAVING_PER_KG": -760.8857730115999,
        "PM_PRICE_RP_PER_KG": 74.7897848595967,
        "PM_PORTO_SAVING_PER_KG": -43.58483013488169,
        "OPERATIONAL_COST_RP_PER_KG": 585.679768339234
     }
  ]
df = pd.DataFrame(data)

# Define the chart generation code as a multi-line string
# NOTE: This code uses the provided DataFrame (df).
# The data values in the df (e.g., RM_PRICE_RP_PER_KG: 313.8)
# differ from the values in the chart image (e.g., -400).
# The code below correctly uses the df values as instructed,
# resulting in a chart with the same structure and colors
# but different bar heights/directions than the image.
chart_code = '''
# Create the figure
fig = go.Figure()

# Get x-axis categories from the DataFrame
x_data = df['GRANULARITY'].astype(str)

# Define colors based on the chart image
colors = {
    'BD_SAVING_ACT_PER_KG': '#FFEB3B',     # Yellow
    'PM_PORTO_SAVINGS_RP_PER_KG': '#AED581', # Light Green
    'PM_PRICE_RP_PER_KG': '#5D6D7E',       # Dark Green/Grey
    'RM_PORTO_SAVINGS_RP_PER_KG': '#E57373', # Pink/Salmon
    'RM_PRICE_RP_PER_KG': '#D32F2F',       # Dark Red
    'OPERATIONAL_COST_RP_PER_KG': '#26A69A'  # Bright Green/Teal
}

# --- Add Traces ---
# The chart stacks positive and negative values separately from zero.
# This is achieved with barmode='stack' (or 'relative', which is equivalent).

# RM_Price_Rp/Kg (Mapped to RM_PRICE_RP_PER_KG)
fig.add_trace(go.Bar(
    name='RM_Price_Rp/Kg',
    x=x_data,
    y=df['RM_PRICE_RP_PER_KG'],
    marker_color=colors['RM_PRICE_RP_PER_KG'],
    text=df['RM_PRICE_RP_PER_KG'].round(0).astype(int),
    textposition='inside',
    texttemplate='%{text}',
    insidetextfont=dict(color='white')
))

# Operational_Cost_Rp/Kg (Mapped to OPERATIONAL_COST_RP_PER_KG)
fig.add_trace(go.Bar(
    name='Operational_Cost_Rp/Kg',
    x=x_data,
    y=df['OPERATIONAL_COST_RP_PER_KG'],
    marker_color=colors['OPERATIONAL_COST_RP_PER_KG'],
    text=df['OPERATIONAL_COST_RP_PER_KG'].round(0).astype(int),
    textposition='inside',
    texttemplate='%{text}',
    insidetextfont=dict(color='white')
))

# RM_Porto_Savings_Rp/Kg (Mapped to RM_PORTO_SAVING_PER_KG)
fig.add_trace(go.Bar(
    name='RM_Porto_Savings_Rp/Kg',
    x=x_data,
    y=df['RM_PORTO_SAVING_PER_KG'],
    marker_color=colors['RM_PORTO_SAVINGS_RP_PER_KG'],
    text=df['RM_PORTO_SAVING_PER_KG'].round(0).astype(int),
    textposition='inside',
    texttemplate='%{text}',
    insidetextfont=dict(color='white')
))

# PM_Price_Rp/Kg (Mapped to PM_PRICE_RP_PER_KG)
fig.add_trace(go.Bar(
    name='PM_Price_Rp/Kg',
    x=x_data,
    y=df['PM_PRICE_RP_PER_KG'],
    marker_color=colors['PM_PRICE_RP_PER_KG'],
    text=df['PM_PRICE_RP_PER_KG'].round(0).astype(int),
    textposition='inside',
    texttemplate='%{text}',
    insidetextfont=dict(color='white')
))

# BD_Saving_Act_Rp/Kg (Mapped to BD_SAVING_ACT_PER_KG)
fig.add_trace(go.Bar(
    name='BD_Saving_Act_Rp/Kg',
    x=x_data,
    y=df['BD_SAVING_ACT_PER_KG'],
    marker_color=colors['BD_SAVING_ACT_PER_KG'],
    text=df['BD_SAVING_ACT_PER_KG'].round(0).astype(int),
    textposition='inside',
    texttemplate='%{text}',
    insidetextfont=dict(color='black')
))

# PM_Porto_Savings_Rp/Kg (Mapped to PM_PORTO_SAVING_PER_KG)
fig.add_trace(go.Bar(
    name='PM_Porto_Savings_Rp/Kg',
    x=x_data,
    y=df['PM_PORTO_SAVING_PER_KG'],
    marker_color=colors['PM_PORTO_SAVINGS_RP_PER_KG'],
    text=df['PM_PORTO_SAVING_PER_KG'].round(0).astype(int),
    textposition='inside',
    texttemplate='%{text}',
    insidetextfont=dict(color='black')
))


# --- Update Layout ---
fig.update_layout(
    title_text='Breakdown saving/loss (Rp./kg) EOM 2023 vs FY 2022',
    title_x=0.5,
    barmode='stack',  # Stacks positive on positive, negative on negative
    xaxis_tickangle=0,
    xaxis=dict(
        title=None,
        categoryorder='array',
        categoryarray=list(x_data), # Use order from df
        showline=True,
        mirror=True,
        linecolor='black',
        ticks='outside'
    ),
    yaxis=dict(
        title=None,
        showgrid=True,
        gridcolor='rgb(230, 230, 230)',
        showline=True,
        mirror=True,
        linecolor='black',
        zeroline=True,
        zerolinecolor='black',
        zerolinewidth=2
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(
        orientation='v',
        yanchor='top',
        y=1.02,
        xanchor='right',
        x=1.15 # Place legend to the right
    ),
    margin=dict(t=100, b=50, l=50, r=250) # Add right margin for legend
)
'''

# --- JSON Configuration and File Saving ---

# 1. Base64-encode the chart code
raw_code = base64.b64encode(chart_code.encode('utf-8')).decode('utf-8')

# 2. Serialize and Base64-encode the DataFrame
df_serialized = df.to_json(orient='records')
df_b64 = base64.b64encode(df_serialized.encode('utf-8')).decode('utf-8')

# 3. Create the JSON configuration object
config = {
    "params": {},
    "raw_code": raw_code,
    "df": df_b64
}

# 4. Save the JSON object to a file
script_name = os.path.splitext(os.path.basename(__file__))[0] if '__file__' in locals() else 'chart_config'
json_file_name = f"{script_name}.json"

with open(json_file_name, 'w') as f:
    json.dump(config, f, indent=4)

print(f"Chart configuration saved to {json_file_name}")

# --- Chart Generation Function ---

def generate_chart_from_config(config, df):
    """
    Decodes and executes the chart code from the config object.
    
    Args:
        config (dict): The configuration dictionary containing 'raw_code'.
        df (pd.DataFrame): The DataFrame to be used by the chart code.
        
    Returns:
        plotly.graph_objects.Figure: The generated Plotly figure.
    """
    # 1. Decode the base64 chart code
    chart_code_decoded = base64.b64decode(config['raw_code']).decode('utf-8')
    
    # 2. Prepare the local scope for exec()
    # This scope will hold the variables the executed code needs
    # (go, pd, df) and the variable it creates (fig).
    local_scope = {
        "go": go,
        "pd": pd,
        "df": df,
        "fig": None  # Initialize fig to None
    }
    
    # 3. Execute the decoded code string
    # The exec() function runs the string as Python code,
    # modifying the local_scope dictionary in place.
    exec(chart_code_decoded, local_scope)
    
    # 4. Return the 'fig' object created by the executed code
    return local_scope['fig']

# --- Render the Chart ---

# Call the function to generate the chart
fig = generate_chart_from_config(config, df)

# Display the chart
fig.show()

Chart configuration saved to chart_config.json


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed